In [41]:
sys.path.append("..")
import os
import utils
import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [42]:
dir_path = "../"
relative_path = "results/500_simulations_contact_based_2021_05_30_23_24_11"
folder_path = dir_path + relative_path
div_filepath = folder_path + "/div.csv"
S_filepath = folder_path + "/S.csv" 
I_filepath = folder_path + "/I.csv" 
new_infected_filepath = folder_path + "/new_infected.csv" 
new_deaths_filepath = folder_path + "/new_deaths.csv" 
vaccinated_filepath = folder_path + "/vaccinated.csv"
pop_age_info_filepath = folder_path + "/start_date_population_age_labels.pkl"

start_date, population, age_labels = utils.read_pickle(pop_age_info_filepath)
paths = utils.create_named_tuple('paths', 'filepaths.txt')
config = utils.create_named_tuple('config', paths.config)

div_df = pd.read_csv(div_filepath, index_col=0)
S_df = pd.read_csv(S_filepath, index_col=0)
I_df = pd.read_csv(I_filepath, index_col=0)
new_infected_df = pd.read_csv(new_infected_filepath, index_col=0)
new_deaths_df = pd.read_csv(new_deaths_filepath, index_col=0)
vaccinated_df = pd.read_csv(vaccinated_filepath, index_col=0)

In [43]:
div_df.head()

,date,simulation_nr,week_nr,vaccines_available,flow_scale,contact_weight_1,contact_weight_2,contact_weight_3,contact_weight_4
0,2020-02-24,1,1,0.0,1.0,0.31,0.24,0.16,0.29
1,2020-03-02,1,2,0.0,1.0,0.31,0.24,0.16,0.29
2,2020-03-09,1,3,0.0,1.0,0.31,0.24,0.16,0.29
3,2020-03-16,1,4,0.0,1.0,0.31,0.24,0.16,0.29
4,2020-03-23,1,5,0.0,1.0,0.31,0.24,0.16,0.29


In [44]:
nr_simulations = div_df.loc[len(div_df)-1].simulation_nr
nr_weeks = div_df.loc[len(div_df)-1].week_nr

In [45]:
vaccines_available = np.zeros((nr_simulations, nr_weeks))
flow_scale = np.zeros((nr_simulations, nr_weeks))
contact_weights = np.zeros((nr_simulations, nr_weeks, 4))
for i in tqdm(range(nr_simulations)):
    df = div_df[div_df.simulation_nr == i+1]
    for j in range(nr_weeks):
        vaccines_available[i, j] = df[df.week_nr == j+1].vaccines_available
        flow_scale[i, j] = df[df.week_nr == j+1].flow_scale
        for k in range(4):
            contact_weights[i, j, k] = df[df.week_nr == j+1][f"contact_weight_{k+1}"]



100%|██████████| 500/500 [02:46<00:00,  3.00it/s]


In [46]:
S_df.head()

,date,simulation_nr,week_nr,S_region_1,S_region_2,S_region_3,S_region_4,S_region_5,S_region_6,S_region_7,...,S_region_354,S_region_355,S_region_356,S_age_groups_1,S_age_groups_2,S_age_groups_3,S_age_groups_4,S_age_groups_5,S_age_groups_6,S_age_groups_7
0,2020-02-24,1,1,696780.0,14784.0,144113.0,37312.0,80445.0,3256.0,3174.0,...,879.0,2200.0,10101.0,1111679.0,1918308.0,746630.0,648962.0,539980.0,308275.0,116175.0
1,2020-03-02,1,2,696780.0,14784.0,144111.0,37312.0,80445.0,3256.0,3174.0,...,874.0,2200.0,10101.0,1111625.0,1918277.0,746613.0,648938.0,539970.0,308269.0,116175.0
2,2020-03-09,1,3,696780.0,14783.0,144106.0,37312.0,80445.0,3256.0,3174.0,...,866.0,2200.0,10100.0,1111536.0,1918208.0,746569.0,648900.0,539951.0,308261.0,116171.0
3,2020-03-16,1,4,696778.0,14781.0,144099.0,37312.0,80445.0,3256.0,3174.0,...,846.0,2200.0,10086.0,1111365.0,1918090.0,746518.0,648822.0,539905.0,308245.0,116166.0
4,2020-03-23,1,5,696773.0,14781.0,144089.0,37310.0,80441.0,3256.0,3174.0,...,829.0,2200.0,10070.0,1111123.0,1917923.0,746415.0,648703.0,539850.0,308217.0,116156.0


In [40]:
nr_regions = len(population)
nr_age_groups = len(age_labels)

S = np.zeros((nr_simulations, nr_weeks, nr_regions, nr_age_groups))
I = np.zeros((nr_simulations, nr_weeks, nr_regions, nr_age_groups))
new_infected = np.zeros((nr_simulations, nr_weeks, nr_regions, nr_age_groups))
new_deaths = np.zeros((nr_simulations, nr_weeks, nr_regions, nr_age_groups))
vaccinated = np.zeros((nr_simulations, nr_weeks, nr_regions, nr_age_groups))

for i in tqdm(range(nr_simulations)):
    S_temp_df = S_df[S_df.simulation_nr == i+1]
    I_temp_df = I_df[I_df.simulation_nr == i+1]
    new_infected_temp_df = new_infected_df[new_infected_df.simulation_nr == i+1]
    new_deaths_temp_df = new_deaths_df[new_deaths_df.simulation_nr == i+1]
    vaccinated_temp_df = vaccinated_df[vaccinated_df.simulation_nr == i+1]    
    for j in range(nr_weeks):
        for k in range(nr_regions):
            S[i, j, k] = S_temp_df[S_temp_df.week_nr == j+1][f"S_region_{k+1}"]
            I[i, j, k] = I_temp_df[I_temp_df.week_nr == j+1][f"I_region_{k+1}"]
            new_infected[i, j, k] = new_infected_temp_df[new_infected_temp_df.week_nr == j+1][f"new_infected_region_{k+1}"]
            new_deaths[i, j, k] = new_deaths_temp_df[new_deaths_temp_df.week_nr == j+1][f"new_deaths_region_{k+1}"]
            vaccinated[i, j, k] = vaccinated_temp_df[vaccinated_temp_df.week_nr == j+1][f"vaccinated_region_{k+1}"]
        for k in range(nr_age_groups):
            S[i, j, k] = S_temp_df[S_temp_df.week_nr == j+1][f"S_age_groups_{k+1}"]
            I[i, j, k] = I_temp_df[I_temp_df.week_nr == j+1][f"I_age_groups_{k+1}"]
            new_infected[i, j, k] = new_infected_temp_df[new_infected_temp_df.week_nr == j+1][f"new_infected_age_groups_{k+1}"]
            new_deaths[i, j, k] = new_deaths_temp_df[new_deaths_temp_df.week_nr == j+1][f"new_deaths_age_groups_{k+1}"]
            vaccinated[i, j, k] = vaccinated_temp_df[vaccinated_temp_df.week_nr == j+1][f"vaccinated_age_groups_{k+1}"]


KeyboardInterrupt: 